We started our journey by analysing our dataset ODIR-2019 
- https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k/data
- https://odir2019.grand-challenge.org/dataset/

From the dataset kaggle page:

> Ocular Disease Intelligent Recognition (ODIR) is a structured ophthalmic database of 5,000 patients with age, 
> color fundus photographs from left and right eyes and doctors' diagnostic keywords from doctors.

> This dataset is meant to represent ‘‘real-life’’ set of patient information collected by Shanggong Medical Technology Co., Ltd. from different hospitals/medical centers in China. In these institutions, fundus images are captured by various cameras in the market, such as Canon, Zeiss and Kowa, resulting into varied image resolutions. Annotations were labeled by trained human readers with quality control management. They classify patient into eight labels including:

> Normal (N),
> Diabetes (D),
> Glaucoma (G),
> Cataract (C),
> Age related Macular Degeneration (A),
> Hypertension (H),
> Pathological Myopia (M),
> Other diseases/abnormalities (O)

Also from the ODIR-2019 grand challenge website:

> The 5,000 patients in this challenge are divided into training, off-site testing and on-site testing subsets. Almost 4,000 cases are used in training stage while  others are for testing stages (off-site and on-site). Table 2 shows the distribution of case number with respect to eight labels in different stages. **Note: one patient may contains one or multiple labels.**

We realise that we have multi-label classification dataset

In [34]:
import numpy as np
import pandas as pd
from PIL import Image
data_set_path = 'ODIR-2019/dataset/full_df.csv'
data_set_img_path = 'ODIR-2019/dataset/preprocessed_images/'

Let us explore the dataset csv or index file to understand our data

In [35]:
df = pd.read_csv(data_set_path)
df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6392 entries, 0 to 6391
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ID                         6392 non-null   int64 
 1   Patient Age                6392 non-null   int64 
 2   Patient Sex                6392 non-null   object
 3   Left-Fundus                6392 non-null   object
 4   Right-Fundus               6392 non-null   object
 5   Left-Diagnostic Keywords   6392 non-null   object
 6   Right-Diagnostic Keywords  6392 non-null   object
 7   N                          6392 non-null   int64 
 8   D                          6392 non-null   int64 
 9   G                          6392 non-null   int64 
 10  C                          6392 non-null   int64 
 11  A                          6392 non-null   int64 
 12  H                          6392 non-null   int64 
 13  M                          6392 non-null   int64 
 14  O       

Let's convert label to python array

We need to simplify this table given there is information for both left and right eyes for the same patient ID in the same row.
Let's explore by viewing patient ID 0

In [37]:
df[df.ID==0]

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
3194,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['C'],"[0, 0, 0, 1, 0, 0, 0, 0]",0_left.jpg


The 2 rows for patient with ID 0 shows how the column relate to the each eye image. We should use filename as unique ID for each fundus file image, labels indicate whether patient has eye problem like cataract or normal. Let's check what are possible values for labels column.

In [38]:
df.labels.value_counts()

labels
['N']    2873
['D']    1608
['O']     708
['C']     293
['G']     284
['A']     266
['M']     232
['H']     128
Name: count, dtype: int64

From label value, we realize that each fundus image only have 1 specific label. Hence, this is still a multi-class classification problem which make our problem simpler as we just need to assign 1 label to each image.

Let's simplify our data and remove factor like Age and Gender from our data.

In [39]:
from ast import literal_eval
df2= df[['filename', 'labels', 'ID', 'Left-Diagnostic Keywords','Right-Diagnostic Keywords']].copy()
df2['labels'] = df2['labels'].apply(literal_eval)
df2['labels'] = df2['labels'].apply(lambda x: x[0])
df2.rename(columns={'labels':'label'}, inplace=True)
map_label = {
    'N': 'Normal',
    'D': 'Diabetes',
    'G': 'Glaucoma',
    'C': 'Cataract',
    'A': 'Ageing',
    'H': 'Hypertension',
    'M': 'Myopia',
    'O': 'Other diseases'
}
df2['class'] = df2['label'].map(map_label)
df2['lr'] = df2['filename'].apply(lambda f: f.split('_')[-1].split('.')[0])
df2['keywords']=np.where(df2['lr']=='right', df2['Right-Diagnostic Keywords'],df2['Left-Diagnostic Keywords'])
df2['keywords']=df2['keywords'].str.replace(',',' ')
df2['keywords']=df2['keywords'].str.replace('，',' ')
df2.head()

,filename,label,ID,Left-Diagnostic Keywords,Right-Diagnostic Keywords,class,lr,keywords
0,0_right.jpg,N,0,cataract,normal fundus,Normal,right,normal fundus
1,1_right.jpg,N,1,normal fundus,normal fundus,Normal,right,normal fundus
2,2_right.jpg,D,2,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,Diabetes,right,moderate non proliferative retinopathy
3,4_right.jpg,D,4,macular epiretinal membrane,mild nonproliferative retinopathy,Diabetes,right,mild nonproliferative retinopathy
4,5_right.jpg,D,5,moderate non proliferative retinopathy,moderate non proliferative retinopathy,Diabetes,right,moderate non proliferative retinopathy


we are aware of 2 possible noises "lens dust" and "optic disk photographically invisible" which may lead to wrong analysis
- lens dust : could be dust in camera or eyes
- optic disk photographically invisible : poor image where optic disc cannot be identified indicating low quality image
Let's check our data

In [40]:
print("optic disk photographically invisible:")
print("\t-current", len(df2[df2['keywords']=='optic disk photographically invisible']))
print("\t-left", len(df2[df2['Left-Diagnostic Keywords']=='optic disk photographically invisible']))
print("\t-right", len(df2[df2['Right-Diagnostic Keywords']=='optic disk photographically invisible']))
print("lens dust:")
print("\t-current", len(df2[df2['keywords'].str.contains("lens dust")]))
print("\t-right", len(df2[df2['Right-Diagnostic Keywords'].str.contains("lens dust")]))
print("\t-left", len(df2[df2['Left-Diagnostic Keywords'].str.contains("lens dust")]))


optic disk photographically invisible:
	-current 0
	-left 3
	-right 1
lens dust:
	-current 109
	-right 131
	-left 158


It seemed the "optic disk photographically invisible" images has already been excluded from our dataset
while "lens dust" still have 109 images.

Nevertheless let us double check "optic disk photographically invisible" by checking filename with left or right diagnostic keywords

In [41]:
df2[(df2['Left-Diagnostic Keywords']=='optic disk photographically invisible')| 
    (df2['Right-Diagnostic Keywords']=='optic disk photographically invisible')].head()

,filename,label,ID,Left-Diagnostic Keywords,Right-Diagnostic Keywords,class,lr,keywords
429,494_right.jpg,D,494,optic disk photographically invisible,moderate non proliferative retinopathy,Diabetes,right,moderate non proliferative retinopathy
1022,1254_right.jpg,G,1254,optic disk photographically invisible,glaucoma,Glaucoma,right,glaucoma
1072,1319_right.jpg,G,1319,optic disk photographically invisible,glaucoma,Glaucoma,right,glaucoma
4281,1369_left.jpg,G,1369,glaucoma,optic disk photographically invisible,Glaucoma,left,glaucoma


It seemed the images no longer contains "optic disk photographically invisible"
Let us remove the "lens dust"

In [42]:
condition_to_keep=~df2['keywords'].str.contains("lens dust")
df2 = df2[condition_to_keep]
print("contains lens dust ", len(df2[df2['keywords'].str.contains("lens dust")]))

contains lens dust  0


lets remove columns that we do not need anymore

In [43]:
df2=df2.drop(columns=['lr','Left-Diagnostic Keywords','Right-Diagnostic Keywords'])
df2.head()

,filename,label,ID,class,keywords
0,0_right.jpg,N,0,Normal,normal fundus
1,1_right.jpg,N,1,Normal,normal fundus
2,2_right.jpg,D,2,Diabetes,moderate non proliferative retinopathy
3,4_right.jpg,D,4,Diabetes,mild nonproliferative retinopathy
4,5_right.jpg,D,5,Diabetes,moderate non proliferative retinopathy


Let us also check image sizes

In [44]:
def get_image_dimensions(image_path):
    """
    Gets the width and height of an image without loading the full raster data.

    Args:
        image_path (str): The path to the image file.

    Returns:
        tuple: A tuple containing (width, height) in pixels.
    """
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            return width, height
    except IOError:
        print(f"Cannot open image file: {image_path}")
        return None, None
    
for index, row in df2.iterrows():
    image_path = data_set_img_path + row['filename']
    width, height = get_image_dimensions(image_path)
    index = hash(index)
    df2.at[index, 'width'] = width
    df2.at[index, 'height'] = height
    df2.at[index, 'wxh']  = f"{width}x{height}"

df2.head()

,filename,label,ID,class,keywords,width,height,wxh
0,0_right.jpg,N,0,Normal,normal fundus,512.0,512.0,512x512
1,1_right.jpg,N,1,Normal,normal fundus,512.0,512.0,512x512
2,2_right.jpg,D,2,Diabetes,moderate non proliferative retinopathy,512.0,512.0,512x512
3,4_right.jpg,D,4,Diabetes,mild nonproliferative retinopathy,512.0,512.0,512x512
4,5_right.jpg,D,5,Diabetes,moderate non proliferative retinopathy,512.0,512.0,512x512


In [45]:
df2.wxh.value_counts()

wxh
512x512    6283
Name: count, dtype: int64

Given all images have the same size, we can remove size columns

In [46]:
df2=df2.drop(columns=['wxh','width','height'])

In [47]:
df2.label.value_counts()

label
N    2818
D    1601
O     681
G     281
C     279
A     266
M     229
H     128
Name: count, dtype: int64

In [48]:
from sklearn.model_selection import train_test_split
y=df2['label']
X=df2.loc[:, df2.columns != 'label']
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42, shuffle=True, stratify=y)
print(f'Original DataFrame shape: {df2.shape}')
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')
print("Train Data Set Output")
print(y_train.value_counts(normalize=True))
print("Test Data Set Output")
print(y_test.value_counts(normalize=True))

Original DataFrame shape: (6283, 5)
X_train shape: (5026, 4)
X_test shape: (1257, 4)
y_train shape: (5026,)
y_test shape: (1257,)
Train Data Set Output
label
N    0.448468
D    0.254875
O    0.108436
G    0.044767
C    0.044369
A    0.042380
M    0.036411
H    0.020294
Name: proportion, dtype: float64
Test Data Set Output
label
N    0.448687
D    0.254574
O    0.108194
G    0.044551
C    0.044551
A    0.042164
M    0.036595
H    0.020684
Name: proportion, dtype: float64


In [49]:
import os 
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

result_datapath="ODIR-2019/results"
os.makedirs(result_datapath, exist_ok=True)
df2.to_csv(os.path.join(result_datapath, '01_eda_df.csv'), index=False)
train_df.to_csv(os.path.join(result_datapath, '01_eda_train_df.csv'), index=False)
test_df.to_csv(os.path.join(result_datapath, '01_eda_test_df.csv'), index=False)